## 1️⃣ Setup & Mount Drive

In [ ]:
# Clone repository
!git clone https://github.com/dungdinhhaha/AIDetect.git
%cd AIDetect

In [ ]:
# Install dependencies
!pip install -q tensorflow==2.19.0 matplotlib seaborn scikit-learn

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Check GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

## 2️⃣ Load Model

In [ ]:
# List available models
import os
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/comparison_detector_models_v2')

print("📁 Available models:")
print("-" * 60)
for f in model_dir.glob('*.h5'):
    size_mb = f.stat().st_size / (1024*1024)
    print(f"  {f.name}: {size_mb:.1f} MB")

for f in model_dir.glob('*.keras'):
    size_mb = f.stat().st_size / (1024*1024)
    print(f"  {f.name}: {size_mb:.1f} MB")

print("-" * 60)

In [ ]:
# Load best model
model_path = '/content/drive/MyDrive/comparison_detector_models_v2/best_model.h5'
# Hoặc dùng final model:
# model_path = '/content/drive/MyDrive/comparison_detector_models_v2/final_model.keras'

print(f"📦 Loading model: {model_path}")
model = tf.keras.models.load_model(model_path)
print("✅ Model loaded!")
print(f"   Input shape: {model.input_shape}")
print(f"   Output shape: {model.output_shape}")

In [ ]:
# Model summary (optional)
model.summary()

## 3️⃣ Load Test Dataset

In [ ]:
from data.loader_tf2 import build_dataset
from configs.config_v2 import IMAGE_SIZE, NUM_CLASSES, BATCH_SIZE

# Load test dataset
test_paths = ['/content/drive/MyDrive/content/data/tct/test.tfrecord']

print(f"📊 Loading test dataset from: {test_paths[0]}")
test_ds = build_dataset(
    test_paths,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    is_training=False
)

# Extract labels for classification
def extract_label(img, tgt):
    return img, tgt['labels'][:, 0]

test_ds_labeled = test_ds.map(extract_label)

print("✅ Test dataset loaded!")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Image size: {IMAGE_SIZE}")
print(f"   Num classes: {NUM_CLASSES}")

## 4️⃣ Quick Evaluation

In [ ]:
# Evaluate on test set (100 batches)
print("⚙️  Evaluating model on test set...")
print("This may take a few minutes...\n")

results = model.evaluate(test_ds_labeled.take(100), verbose=1)

test_loss = results[0]
test_accuracy = results[1]

print("\n" + "=" * 60)
print("📊 TEST RESULTS")
print("=" * 60)
print(f"Test Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("=" * 60)

## 5️⃣ Detailed Analysis with Predictions

In [ ]:
import numpy as np

# Collect all predictions and ground truth
print("🔮 Collecting predictions...")

all_predictions = []
all_true_labels = []
all_images = []

num_batches = 100  # Test on 100 batches

for i, (images, labels) in enumerate(test_ds_labeled.take(num_batches)):
    if i % 10 == 0:
        print(f"  Processing batch {i}/{num_batches}...")
    
    preds = model.predict(images, verbose=0)
    all_predictions.extend(np.argmax(preds, axis=1))
    all_true_labels.extend(labels.numpy().astype(int))
    all_images.extend(images.numpy())

all_predictions = np.array(all_predictions)
all_true_labels = np.array(all_true_labels)
all_images = np.array(all_images)

print(f"\n✅ Processed {len(all_predictions)} images")

In [ ]:
# Calculate accuracy
accuracy = np.mean(all_predictions == all_true_labels)
print(f"\n📊 Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Correct: {np.sum(all_predictions == all_true_labels)}")
print(f"   Incorrect: {np.sum(all_predictions != all_true_labels)}")

## 6️⃣ Classification Report

In [ ]:
from sklearn.metrics import classification_report

print("=" * 60)
print("📋 CLASSIFICATION REPORT")
print("=" * 60)
print(classification_report(
    all_true_labels,
    all_predictions,
    target_names=[f"Class {i}" for i in range(NUM_CLASSES)],
    zero_division=0
))
print("=" * 60)

## 7️⃣ Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate confusion matrix
cm = confusion_matrix(all_true_labels, all_predictions)

# Plot
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f"C{i}" for i in range(NUM_CLASSES)],
            yticklabels=[f"C{i}" for i in range(NUM_CLASSES)])
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Save to Drive
save_path = '/content/drive/MyDrive/comparison_detector_models_v2/confusion_matrix.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"\n✅ Confusion matrix saved to: {save_path}")

## 8️⃣ Visualize Sample Predictions

In [ ]:
# Get correct and incorrect predictions
correct_mask = all_predictions == all_true_labels
incorrect_mask = ~correct_mask

correct_indices = np.where(correct_mask)[0]
incorrect_indices = np.where(incorrect_mask)[0]

print(f"✅ Correct predictions: {len(correct_indices)}")
print(f"❌ Incorrect predictions: {len(incorrect_indices)}")

In [ ]:
# Visualize CORRECT predictions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('✅ CORRECT PREDICTIONS (Sample)', fontsize=16, color='green')

for i, ax in enumerate(axes.flat):
    if i < len(correct_indices):
        idx = correct_indices[i]
        ax.imshow(all_images[idx])
        ax.set_title(f"True: {all_true_labels[idx]}, Pred: {all_predictions[idx]}",
                    color='green', fontsize=10, fontweight='bold')
        ax.axis('off')

plt.tight_layout()
plt.show()

# Save
save_path = '/content/drive/MyDrive/comparison_detector_models_v2/correct_predictions.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✅ Saved to: {save_path}")

In [ ]:
# Visualize INCORRECT predictions
if len(incorrect_indices) > 0:
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('❌ INCORRECT PREDICTIONS (Sample)', fontsize=16, color='red')

    for i, ax in enumerate(axes.flat):
        if i < min(8, len(incorrect_indices)):
            idx = incorrect_indices[i]
            ax.imshow(all_images[idx])
            ax.set_title(f"True: {all_true_labels[idx]}, Pred: {all_predictions[idx]}",
                        color='red', fontsize=10, fontweight='bold')
            ax.axis('off')

    plt.tight_layout()
    plt.show()

    # Save
    save_path = '/content/drive/MyDrive/comparison_detector_models_v2/incorrect_predictions.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"✅ Saved to: {save_path}")
else:
    print("🎉 Perfect accuracy! No incorrect predictions!")

## 9️⃣ Class Distribution Analysis

In [ ]:
# Plot class distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# True labels distribution
unique_true, counts_true = np.unique(all_true_labels, return_counts=True)
ax1.bar(unique_true, counts_true, color='skyblue', edgecolor='black')
ax1.set_title('True Label Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Class', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# Predicted labels distribution
unique_pred, counts_pred = np.unique(all_predictions, return_counts=True)
ax2.bar(unique_pred, counts_pred, color='salmon', edgecolor='black')
ax2.set_title('Predicted Label Distribution', fontsize=14, fontweight='bold')
ax2.set_xlabel('Class', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Save
save_path = '/content/drive/MyDrive/comparison_detector_models_v2/class_distribution.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✅ Saved to: {save_path}")

## 🔟 Per-Class Accuracy

In [ ]:
# Calculate per-class accuracy
print("=" * 60)
print("📊 PER-CLASS ACCURACY")
print("=" * 60)

class_accuracies = []
class_labels = []

for i in range(NUM_CLASSES):
    mask = all_true_labels == i
    if np.sum(mask) > 0:
        class_acc = np.mean(all_predictions[mask] == all_true_labels[mask])
        total = np.sum(mask)
        correct = np.sum(all_predictions[mask] == all_true_labels[mask])
        
        class_accuracies.append(class_acc)
        class_labels.append(i)
        
        print(f"Class {i:2d}: {class_acc:.4f} ({class_acc*100:5.2f}%) - {correct}/{total} correct")

print("=" * 60)

In [ ]:
# Plot per-class accuracy
plt.figure(figsize=(12, 6))
bars = plt.bar(class_labels, class_accuracies, color='teal', edgecolor='black', alpha=0.7)

# Color bars based on accuracy
for i, (bar, acc) in enumerate(zip(bars, class_accuracies)):
    if acc >= 0.9:
        bar.set_color('green')
    elif acc >= 0.7:
        bar.set_color('orange')
    else:
        bar.set_color('red')

plt.title('Per-Class Accuracy', fontsize=16, fontweight='bold')
plt.xlabel('Class', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.ylim(0, 1.05)
plt.grid(axis='y', alpha=0.3)
plt.axhline(y=accuracy, color='black', linestyle='--', linewidth=2, label=f'Overall: {accuracy:.3f}')
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Save
save_path = '/content/drive/MyDrive/comparison_detector_models_v2/per_class_accuracy.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✅ Saved to: {save_path}")

## 💾 Save Test Results

In [ ]:
import json
from datetime import datetime

# Prepare results dictionary
results_dict = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_path': model_path,
    'total_images_tested': len(all_predictions),
    'overall_accuracy': float(accuracy),
    'test_loss': float(test_loss),
    'num_correct': int(np.sum(all_predictions == all_true_labels)),
    'num_incorrect': int(np.sum(all_predictions != all_true_labels)),
    'per_class_accuracy': {},
    'confusion_matrix': cm.tolist()
}

# Add per-class accuracy
for i in range(NUM_CLASSES):
    mask = all_true_labels == i
    if np.sum(mask) > 0:
        class_acc = np.mean(all_predictions[mask] == all_true_labels[mask])
        results_dict['per_class_accuracy'][f'class_{i}'] = {
            'accuracy': float(class_acc),
            'total_samples': int(np.sum(mask)),
            'correct_predictions': int(np.sum(all_predictions[mask] == all_true_labels[mask]))
        }

# Save to JSON
save_path = '/content/drive/MyDrive/comparison_detector_models_v2/test_results.json'
with open(save_path, 'w') as f:
    json.dump(results_dict, f, indent=2)

print(f"✅ Test results saved to: {save_path}")
print(f"\n📄 Results preview:")
print(json.dumps(results_dict, indent=2)[:500] + "...")

## 📊 Final Summary

In [ ]:
print("\n" + "=" * 60)
print("✅ TESTING COMPLETED!")
print("=" * 60)
print(f"\n📊 Summary:")
print(f"  Model: {Path(model_path).name}")
print(f"  Images tested: {len(all_predictions)}")
print(f"  Overall accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  Test loss: {test_loss:.4f}")
print(f"\n📁 Files saved in Drive:")
print(f"  - confusion_matrix.png")
print(f"  - correct_predictions.png")
if len(incorrect_indices) > 0:
    print(f"  - incorrect_predictions.png")
print(f"  - class_distribution.png")
print(f"  - per_class_accuracy.png")
print(f"  - test_results.json")
print("\n" + "=" * 60)
print("🎉 All visualizations and results saved to Google Drive!")
print("=" * 60)